In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from skstats.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from scipy import stats

### Reading the data files.

In [2]:
train_2016 = pd.read_csv('data/train_2016_v2.csv')

In [3]:
properties_2016 = pd.read_csv('data/properties_2016.csv')

/home/tyagi/CSE519-2017-111482304-111733554/env/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train_2017 = pd.read_csv('data/train_2017.csv')

In [5]:
properties_2017 = pd.read_csv('data/properties_2017.csv')

/home/tyagi/CSE519-2017-111482304-111733554/env/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Reducing the size of the dataframes

#### Setting up the functions

In [6]:
def get_min_memory_type(colobj, verbose=0):
    orig_type = colobj.dtype
    new_types = ["uint8", "int8", "uint16", "int16", 'float16', "uint32", "int32", 'float32']
    if (orig_type not in new_types and orig_type not in ["int64", "uint64", "float64"]):
        if (verbose > 0):
            print("Type {} of column {} is unsupported".format(orig_type, colobj.name))
        return orig_type

    for new_type in new_types:
        yes_min = colobj.min() == colobj.min().astype(new_type).astype(orig_type)
        yes_max = colobj.max() == colobj.max().astype(new_type).astype(orig_type)
        if (yes_min and yes_max):
            return new_type

    if (verbose > 0):
        print("No optimisation found for {} column of {} type".format(colobj.name, orig_type))
    return orig_type


def reduce_mem_usage(df, exclude=[], verbose=0):
    start_mem_usg = df.memory_usage().sum() / 1024 ** 2
    if (verbose > 0):
        print("Memory usage before :", start_mem_usg, " MB")

    for colname in df.columns:
        if (colname not in exclude):
            colobj = getattr(df, colname)
            new_type = get_min_memory_type(colobj, verbose)
            if (new_type != colobj.dtype):
                if (verbose > 0):
                    print("Converting {} column from {} to {}".format(colname, colobj.dtype, new_type))
                df[colname] = df[colname].astype(new_type)

    end_mem_usg = df.memory_usage().sum() / 1024 ** 2

    if (verbose > 0):
        print("Memory usage after :", end_mem_usg, " MB")

    mem_usg_diff_prc = float((end_mem_usg - start_mem_usg)) * 100 / start_mem_usg

    if (verbose > 0):
        if (end_mem_usg < start_mem_usg):
            print("Memory gain: {0:0.2f}%".format(-mem_usg_diff_prc))
        else:
            print("Memory loss: {0:0.2f}%".format(mem_usg_diff_prc))

    return -mem_usg_diff_prc

def fillna_mean(df, cols):
    for col in cols:
        mean_values = df[[col]].mean(axis=0)
#         print(col, mean_values[col])
        df[col].fillna(mean_values[col], inplace=True)

#### Filling the Nan with Means

In [7]:
fillna_mean(properties_2016, ['airconditioningtypeid','architecturalstyletypeid','basementsqft',
                        'bathroomcnt','bedroomcnt','buildingqualitytypeid','buildingclasstypeid',
                        'calculatedbathnbr','decktypeid','threequarterbathnbr','finishedfloor1squarefeet',
                        'calculatedfinishedsquarefeet','finishedsquarefeet6','finishedsquarefeet12',
                        'finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                        'fireplacecnt','fireplaceflag','fullbathcnt','garagecarcnt','garagetotalsqft',
                        'hashottuborspa','heatingorsystemtypeid','lotsizesquarefeet',
                        'numberofstories','poolcnt','poolsizesum','pooltypeid10','pooltypeid2',
                        'pooltypeid7','regionidcounty','regionidcity','regionidzip',
                        'regionidneighborhood','roomcnt','storytypeid','typeconstructiontypeid',
                        'unitcnt','yardbuildingsqft17','yardbuildingsqft26','yearbuilt','taxvaluedollarcnt',
                        'structuretaxvaluedollarcnt','landtaxvaluedollarcnt','taxamount','assessmentyear'])

In [8]:
fillna_mean(properties_2017, ['airconditioningtypeid','architecturalstyletypeid','basementsqft',
                        'bathroomcnt','bedroomcnt','buildingqualitytypeid','buildingclasstypeid',
                        'calculatedbathnbr','decktypeid','threequarterbathnbr','finishedfloor1squarefeet',
                        'calculatedfinishedsquarefeet','finishedsquarefeet6','finishedsquarefeet12',
                        'finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                        'fireplacecnt','fireplaceflag','fullbathcnt','garagecarcnt','garagetotalsqft',
                        'hashottuborspa','heatingorsystemtypeid','lotsizesquarefeet',
                        'numberofstories','poolcnt','poolsizesum','pooltypeid10','pooltypeid2',
                        'pooltypeid7','regionidcounty','regionidcity','regionidzip',
                        'regionidneighborhood','roomcnt','storytypeid','typeconstructiontypeid',
                        'unitcnt','yardbuildingsqft17','yardbuildingsqft26','yearbuilt','taxvaluedollarcnt',
                        'structuretaxvaluedollarcnt','landtaxvaluedollarcnt','taxamount','assessmentyear'])

#### Reduce the memory usage for properties file

In [9]:
reduce_mem_usage(properties_2016[['airconditioningtypeid','architecturalstyletypeid','basementsqft',
                        'bathroomcnt','bedroomcnt','buildingqualitytypeid','buildingclasstypeid',
                        'calculatedbathnbr','decktypeid','threequarterbathnbr','finishedfloor1squarefeet',
                        'calculatedfinishedsquarefeet','finishedsquarefeet6','finishedsquarefeet12',
                        'finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                        'fireplacecnt','fireplaceflag','fullbathcnt','garagecarcnt','garagetotalsqft',
                        'hashottuborspa','heatingorsystemtypeid','lotsizesquarefeet',
                        'numberofstories','poolcnt','poolsizesum','pooltypeid10','pooltypeid2',
                        'pooltypeid7','regionidcounty','regionidcity','regionidzip',
                        'regionidneighborhood','roomcnt','storytypeid','typeconstructiontypeid',
                        'unitcnt','yardbuildingsqft17','yardbuildingsqft26','yearbuilt','taxvaluedollarcnt',
                        'structuretaxvaluedollarcnt','landtaxvaluedollarcnt','taxamount','assessmentyear']],
                        [],1)

Memory usage before : 1070.44379425  MB
Converting airconditioningtypeid column from float64 to uint8


/home/tyagi/CSE519-2017-111482304-111733554/env/lib/python3.4/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Converting architecturalstyletypeid column from float64 to uint8
Converting basementsqft column from float64 to uint16
Converting bathroomcnt column from float64 to uint8
Converting bedroomcnt column from float64 to uint8
Converting buildingqualitytypeid column from float64 to uint8
Converting buildingclasstypeid column from float64 to uint8
Converting calculatedbathnbr column from float64 to uint8
Converting decktypeid column from float64 to uint8
Converting threequarterbathnbr column from float64 to uint8
Converting finishedfloor1squarefeet column from float64 to uint16
Converting calculatedfinishedsquarefeet column from float64 to uint32
Converting finishedsquarefeet6 column from float64 to uint32
Converting finishedsquarefeet12 column from float64 to uint32
Converting finishedsquarefeet13 column from float64 to uint16
Converting finishedsquarefeet15 column from float64 to uint32
Converting finishedsquarefeet50 column from float64 to uint16
Converting fireplacecnt column from float6

69.946803525306038

In [10]:
reduce_mem_usage(properties_2017[['airconditioningtypeid','architecturalstyletypeid','basementsqft',
                        'bathroomcnt','bedroomcnt','buildingqualitytypeid','buildingclasstypeid',
                        'calculatedbathnbr','decktypeid','threequarterbathnbr','finishedfloor1squarefeet',
                        'calculatedfinishedsquarefeet','finishedsquarefeet6','finishedsquarefeet12',
                        'finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                        'fireplacecnt','fireplaceflag','fullbathcnt','garagecarcnt','garagetotalsqft',
                        'hashottuborspa','heatingorsystemtypeid','lotsizesquarefeet',
                        'numberofstories','poolcnt','poolsizesum','pooltypeid10','pooltypeid2',
                        'pooltypeid7','regionidcounty','regionidcity','regionidzip',
                        'regionidneighborhood','roomcnt','storytypeid','typeconstructiontypeid',
                        'unitcnt','yardbuildingsqft17','yardbuildingsqft26','yearbuilt','taxvaluedollarcnt',
                        'structuretaxvaluedollarcnt','landtaxvaluedollarcnt','taxamount','assessmentyear']],
                        [],1)

Memory usage before : 1070.44379425  MB
Converting airconditioningtypeid column from float64 to uint8


/home/tyagi/CSE519-2017-111482304-111733554/env/lib/python3.4/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Converting architecturalstyletypeid column from float64 to uint8
Converting basementsqft column from float64 to uint16
Converting bathroomcnt column from float64 to uint8
Converting bedroomcnt column from float64 to uint8
Converting buildingqualitytypeid column from float64 to uint8
Converting buildingclasstypeid column from float64 to uint8
Converting calculatedbathnbr column from float64 to uint8
Converting decktypeid column from float64 to uint8
Converting threequarterbathnbr column from float64 to uint8
Converting finishedfloor1squarefeet column from float64 to uint16
Converting calculatedfinishedsquarefeet column from float64 to uint32
Converting finishedsquarefeet6 column from float64 to uint32
Converting finishedsquarefeet12 column from float64 to uint32
Converting finishedsquarefeet13 column from float64 to uint16
Converting finishedsquarefeet15 column from float64 to uint32
Converting finishedsquarefeet50 column from float64 to uint16
Converting fireplacecnt column from float6

69.946803525306038

### Merging the properties and logerrors

In [11]:
prop16_train16 = pd.merge(train_2016,properties_2016, on='parcelid', how='left')
prop16_train17 = pd.merge(train_2017,properties_2016, on='parcelid', how='left')
prop17_train16 = pd.merge(train_2016,properties_2017, on='parcelid', how='left')
prop17_train17 = pd.merge(train_2017,properties_2017, on='parcelid', how='left')

Use data normalization and not zscore thing.

In [18]:
min_max_scalar = MinMaxScaler()
prop16_train16[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']] = min_max_scalar.fit_transform(
prop16_train16[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']])

In [19]:
prop16_train17[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']] = min_max_scalar.fit_transform(
prop16_train17[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']])
prop17_train16[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']] = min_max_scalar.fit_transform(
prop17_train16[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']])
prop17_train17[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']] = min_max_scalar.fit_transform(
prop17_train17[['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr','threequarterbathnbr',
                'finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet6',
                'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50',
                'fireplacecnt','fullbathcnt','garagecarcnt','garagetotalsqft','lotsizesquarefeet',
                'numberofstories','poolcnt','poolsizesum','roomcnt','unitcnt','yardbuildingsqft17',
                'yardbuildingsqft26','taxvaluedollarcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt',
                'taxamount']])

### Seperating the training and testing files

In [20]:
test1 = prop16_train16.iloc[0:9000]
test2 = prop16_train17.iloc[7000:14000]
test3 = prop17_train16.iloc[18000:27000]
test4 = prop17_train17.iloc[21000:28000]

In [21]:
train1 = prop16_train16.drop(prop16_train16.index[0:9000])
train2 = prop16_train17.drop(prop16_train17.index[7000:14000])
train3 = prop17_train16.drop(prop17_train16.index[18000:27000])
train4 = prop17_train17.drop(prop17_train17.index[21000:28000])

### Now the feature selection
#### Filter the data first

In [57]:
x_train1 = train1.drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode','taxdelinquencyflag','taxdelinquencyyear',
                          'censustractandblock'], axis=1)
x_train2 = train2.drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode','taxdelinquencyflag','taxdelinquencyyear',
                          'censustractandblock'], axis=1)
x_train3 = train3.drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode','taxdelinquencyflag','taxdelinquencyyear',
                          'censustractandblock'], axis=1)
x_train4 = train4.drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode','taxdelinquencyflag','taxdelinquencyyear',
                          'censustractandblock'], axis=1)

### Performing linear regression and Lasso for feature selection

In [58]:
x_train1 = x_train1.dropna(axis=0, how='any')
x_train2 = x_train2.dropna(axis=0, how='any')
x_train3 = x_train3.dropna(axis=0, how='any')
x_train4 = x_train4.dropna(axis=0, how='any')

In [59]:
Y1 = x_train1['logerror'].values
Y2 = x_train2['logerror'].values
Y3 = x_train3['logerror'].values
Y4 = x_train4['logerror'].values
x_train1 = x_train1.drop(['logerror'], axis=1)
x_train2 = x_train2.drop(['logerror'], axis=1)
x_train3 = x_train3.drop(['logerror'], axis=1)
x_train4 = x_train4.drop(['logerror'], axis=1)

In [60]:
X1 = x_train1.as_matrix()
X2 = x_train2.as_matrix()
X3 = x_train3.as_matrix()
X4 = x_train4.as_matrix()
colnames = x_train1.columns

In [50]:
ranks = {}
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [61]:
# Using Linear Regression
lr = LinearRegression()
lr.fit(X1,Y1)
ranks["LinReg1"] = ranking(np.abs(lr.coef_), colnames)
lr.fit(X2,Y2)
ranks["LinReg2"] = ranking(np.abs(lr.coef_), colnames)
lr.fit(X3,Y3)
ranks["LinReg3"] = ranking(np.abs(lr.coef_), colnames)
lr.fit(X4,Y4)
ranks["LinReg4"] = ranking(np.abs(lr.coef_), colnames)


In [62]:
# Using Ridge 
ridge = Ridge(alpha = 7)
ridge.fit(X1,Y1)
ranks['Ridge1'] = ranking(np.abs(ridge.coef_), colnames)
ridge.fit(X2,Y2)
ranks['Ridge2'] = ranking(np.abs(ridge.coef_), colnames)
ridge.fit(X3,Y3)
ranks['Ridge3'] = ranking(np.abs(ridge.coef_), colnames)
ridge.fit(X4,Y4)
ranks['Ridge4'] = ranking(np.abs(ridge.coef_), colnames)


In [63]:
# # Using Lasso
lasso = Lasso(alpha=.05)
lasso.fit(X1, Y1)
ranks["Lasso1"] = ranking(np.abs(lasso.coef_), colnames)
lasso.fit(X2, Y2)
ranks["Lasso2"] = ranking(np.abs(lasso.coef_), colnames)
lasso.fit(X3, Y3)
ranks["Lasso3"] = ranking(np.abs(lasso.coef_), colnames)
lasso.fit(X4, Y4)
ranks["Lasso4"] = ranking(np.abs(lasso.coef_), colnames)


In [64]:
# Random Forest Feature ranking
rf = RandomForestRegressor(n_jobs=-1, n_estimators=50, verbose=2)
rf.fit(X1,Y1)
ranks["RF1"] = ranking(rf.feature_importances_, colnames)
rf.fit(X2,Y2)
ranks["RF1"] = ranking(rf.feature_importances_, colnames)
rf.fit(X3,Y3)
ranks["RF1"] = ranking(rf.feature_importances_, colnames)
rf.fit(X4,Y4)
ranks["RF1"] = ranking(rf.feature_importances_, colnames)

building tree 3 of 50building tree 2 of 50
building tree 4 of 50building tree 1 of 50


building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   59.4s


building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.4min finished


building tree 3 of 50
building tree 1 of 50building tree 4 of 50building tree 2 of 50


building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50


...

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.4min finished


building tree 1 of 50building tree 2 of 50building tree 3 of 50building tree 4 of 50



building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   49.6s


building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.2min finished


In [69]:
# Create empty dictionary to store the mean value calculated from all the scores
r = {}
for name in colnames:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)
 
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")

# print("\t%s" % "\t".join(methods))
# for name in colnames:
#     print("%s\t%s" % (name, "\t".join(map(str, 
#                          [ranks[method][name] for method in methods]))))


In [70]:
# Put the mean scores into a Pandas dataframe
meanplot = pd.DataFrame(list(r.items()), columns= ['Feature','Mean Ranking'])

# Sort the dataframe
meanplot = meanplot.sort_values('Mean Ranking', ascending=False)


In [72]:
meanplot

,Feature,Mean Ranking
5,taxvaluedollarcnt,0.48
27,landtaxvaluedollarcnt,0.42
48,structuretaxvaluedollarcnt,0.34
35,finishedsquarefeet12,0.31
42,taxamount,0.29
38,regionidcounty,0.25
37,calculatedfinishedsquarefeet,0.20
22,lotsizesquarefeet,0.19
6,yearbuilt,0.17
31,regionidzip,0.13
